# FEMpy Demonstration of L-Shaped Bracket

In this demonstration, we have an L-Shaped Bracket that is fixed at the top and has applied loads to its right-hand side. 

First, we import `FEMpy` and any required packages:

In [11]:
import FEMpy as fp
import numpy as np

### Create an Isotropic Plane Stress Constitutive Model

Lets consider 7000 series Aluminum wih elastic modulus $\lambda = 71.7$ $GPa$, Poisson's ratio $\nu = 0.33$, density $\rho = 2780$ $\frac{kg}{m^3}$, and thickness $t = 5$ $mm$.

<img src="Presentation/LBeam.png" width="400">

In [14]:
λ, ν, ρ, t = 71.7e9, 0.33, 2780, 5e-3
constitutiveModel = fp.Constitutive.IsoPlaneStress(λ, ν, ρ, t)

### Create `FEMpy` model  
Load in mesh and add fixed boundary condition. We also specify the output directory name, the output format of the mesh, and Von-Mises-Stresses as a desired output quantity.

The mesh is comprised of 8,184 quad elements

<img src="Presentation/LBeamMesh.png" width="400">

In [15]:
# load in mesh
options = {"outputDir": "ExampleOutput", "outputFormat": ".vtu", "outputFunctions": ["Von-Mises-Stress"]}
model = fp.FEMpyModel(constitutiveModel, meshFileName="Meshes/LBracket.msh", options=options)

# add fixed boundary condition to top edge
nodeCoords = model.getCoordinates()
topEdgeNodeInds = np.argwhere(nodeCoords[:, 1] == np.max(nodeCoords[:, 1])).flatten()

model.addFixedBCToNodes(name="Fixed", nodeInds=topEdgeNodeInds, dof=[0, 1], value=0.0)


Welcome to
 ______ ______ __  __
|  ____|  ____|  \/  |
| |__  | |__  | \  / |_ __  _   _
|  __| |  __| | |\/| | '_ \| | | |
| |    | |____| |  | | |_) | |_| |
|_|    |______|_|  |_| .__/ \__, |
                    | |     __/ |
                    |_|    |___/

Version: 1.0.0




### Set up `FEMpy` problems

Create two problem cases: Vertical Load and Horizonal Load of value $1\times 10^5$

<img src="Presentation/LBeamLoadCases.png" width="500">

In [16]:
# add load cases to FEMpy model
verticalLoadCase = model.addProblem("Vertical-Load", options={"printTiming": True})
horizontalLoadCase = model.addProblem("Horizontal-Load", options={"printTiming": True})

# get right-side node indices, add loads to each problem
rightEdgeNodeInds = np.argwhere(nodeCoords[:, 0] == np.max(nodeCoords[:, 0])).flatten()

verticalLoadCase.addLoadToNodes(name="RightEdgeLoad", nodeInds=rightEdgeNodeInds, dof=1, value=-1e5, totalLoad=True)
horizontalLoadCase.addLoadToNodes(name="RightEdgeLoad", nodeInds=rightEdgeNodeInds, dof=0, value=-1e5, totalLoad=True)

### Solve each problem

In [6]:
for problem in model.problems:
    problem.solve()
    problem.writeSolution()

Updating Residual
Updating Jacobian
Factorising Jacobian
Solving linear system


+-----------------------------------------------------------------+
 Timing information for FEMpy problem: Vertical-Load
+-----------------------------------------------------------------+
+ Residual Assembly: 1.81424e-01 s
+ Jacobian Assembly: 4.89288e-01 s
+ Linear Solution:   1.72724e-01 s
+-----------------------------------------------------------------+

Updating Residual
Updating Jacobian
Factorising Jacobian
Solving linear system


+-----------------------------------------------------------------+
 Timing information for FEMpy problem: Horizontal-Load
+-----------------------------------------------------------------+
+ Residual Assembly: 9.99920e-02 s
+ Jacobian Assembly: 4.54143e-01 s
+ Linear Solution:   1.76414e-01 s
+-----------------------------------------------------------------+



### Results  
The resulting mesh contains the quantities:
* X-displacements
* Y-displacements
* Von-Misen Stress

The figures below show the resulting Von-Misen Stress, and the resulting displacements.

<img src="Presentation/VonMises.png" width="800">